In [1]:
#twitter API dependencies
import os
# from os import path

# Module for reading CSV files
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
import json
from Jacks_config import client_id
from Jacks_config import client_secret
import nasdaqdatalink



 
import urllib.request #for url-encode
import urllib3 as urllib #for getting and reciving data from server
import time #unix timestamp import oauth2

In [ ]:
base_url = "https://api.twitter.com/oauth2/token"

bearer_token = requests.post(base_url, data={"client_id":client_id,
                                           "client_secret":client_secret,
                                           "grant_type": "client_credentials"}).json()
bearer_token

In [ ]:
#put token in format for requests
B_Tok = f"Bearer {bearer_token['access_token']}"
B_Tok

In [ ]:
#get elon's twitter id
headers = {
    'Accept' : '0',
    'Authorization': B_Tok
}

#define first request
E_twitterID = requests.get('https://api.twitter.com/2/users/by/username/elonmusk',
                 params={
                     
                 }, headers=headers).json()
E_twitterID

In [ ]:
#Get Elon's tweets
search_url = 'https://api.twitter.com/2/users/44196397/tweets'
Response = []
headers = {
    'Accept' : '0',
    'Authorization': B_Tok
}

params = {'max_results':100,
          'tweet.fields': 'created_at'
          
}

#define first request
tweet_count = requests.get(search_url,
                           params=params, headers=headers).json()
Response.append(tweet_count)
pprint(tweet_count)

while tweet_count['meta']['next_token']:
    params['pagination_token'] = tweet_count['meta']['next_token']
    tweet_count = requests.get(search_url,
                           params=params, headers=headers).json()
    Response.append(tweet_count)


In [ ]:
#Look at JSON response
Response

In [ ]:
#get number of responses
response_len = len(Response)
response_len


In [ ]:
#save each peice of information we want to a list
tweet_id = []
tweets = []
tweet_date = []



for i in range(0, response_len-1):
    Response[i]
    for x in range(0, 99):
        tweet_id.append(Response[i]['data'][x]['id'])
        tweets.append(Response[i]['data'][x]['text'])
        tweet_date.append(Response[i]['data'][x]['created_at'])
        
        

In [ ]:
#clean the date so it is just the date without time
tweet_date
tweet_date_len = len(tweet_date)
tweet_date_cleaned = []

for i in range(0,tweet_date_len):
    tweet_date_cleaned.append(tweet_date[i][0:10])
tweet_date_cleaned

In [ ]:
#get bool responses on if twitter or tesla was mentioned and save to list

tesla_bool = []
twitter_bool = []

for i in range(0, tweet_date_len):
    tesla_bool.append(("Tesla" or "TSLA") in tweets[i])
    twitter_bool.append(("Twitter" or "TWTR") in tweets[i])

tesla_bool

In [ ]:
#add lists into a dictionary
tweet_info = {'tweet_date': tweet_date_cleaned, 'tweet_text': tweets, 'tesla_mention': tesla_bool, 'twitter_mention': twitter_bool}

In [ ]:
#view in df, problem with this DF is it has multiple tweets for each date
tweet_info_df = pd.DataFrame(tweet_info)
tweet_info_df.head()

In [ ]:
#find unique date values
single_dates = []

df_len = len(tweet_info_df['tweet_date'].unique())

for i in tweet_info_df:
    single_dates.append(tweet_info_df['tweet_date'].unique())
    
single_dates

In [ ]:
#select only first array so I have one list of unique values 
clean_dates = single_dates.pop(1)
clean_dates

In [ ]:
#loop through single dates and insert a true or false statement based on if elon mentions twitter or tesla on given date

cleaned_dates_len = len(clean_dates)
print(cleaned_dates_len)

single_date_tesla_bool = []
single_date_twitter_bool = []

for i in range(0,cleaned_dates_len):
    clean_dates[i]
    for  x in range (0, tweet_date_len):
        if clean_dates[i] == tweet_date_cleaned[x] and tesla_bool[x] == True:
            single_date_tesla_bool.append('True')
            break
        elif clean_dates[i] != tweet_date_cleaned[x] and clean_dates[i] == tweet_date_cleaned[x-1]:
            single_date_tesla_bool.append('False')
            break

for o in range(0,cleaned_dates_len):
    clean_dates[o]
    for  y in range (0, tweet_date_len):
        if clean_dates[o] == tweet_date_cleaned[y] and twitter_bool[y] == True:
            single_date_twitter_bool.append('True')
            break
        elif clean_dates[o] != tweet_date_cleaned[y] and clean_dates[o] == tweet_date_cleaned[y-1]:
            single_date_twitter_bool.append('False')
            break
            

        

In [ ]:
#test to double check length is correct
print(len(single_date_tesla_bool))

In [ ]:
single_date_tesla_bool

In [ ]:
#combine single dates, and mention bool statements into DF

single_date_tweet_info = {'tweet_date': clean_dates, 'tesla_mention': single_date_tesla_bool,
                          'twitter_mention': single_date_twitter_bool}

#convert to df
single_date_df = pd.DataFrame(single_date_tweet_info)
single_date_df

In [ ]:
single_date_df.to_csv('data_files/tweet_mentions.csv', index=False)